In [1]:
import os
from pathlib import Path
import simpy
    
FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day10.txt'

## Part One

**SimPy** is a process-based discrete-event simulation framework based on standard Python:  
https://simpy.readthedocs.io/en/latest/contents.html

In [2]:
def add(env, value):
    yield env.timeout(2)
    return int(value)

def noop(env):
    yield env.timeout(1)
    
def process_instructions(env):
    with open(FOLDER/in_file) as f:
        for line in map(str.strip, f):
            if line != "noop":
                inst, value = line.split()
                env.X += yield env.process(add(env, value))
            else:
                yield env.process(noop(env))

env = simpy.Environment()
env.process(process_instructions(env))
env.X = 1

total = 0
for i in [20, 60, 100, 140, 180, 220]:
    env.run(until=i)
    total += i * env.X
     
    
total

14160

## Part Two

In [112]:
env = simpy.Environment()
env.process(process_instructions(env))
env.X = 0

for row in range(6):
    for sprite_pos in range(40):
        env.run(until=env.now+1)
        
        if env.X + 2 >= sprite_pos >= env.X:
            print('#', end='')
        else:
            print('.', end='')
    print()


###....##.####.###..###..####.####..##..
#..#....#.#....#..#.#..#.#....#....#..#.
#..#....#.###..#..#.#..#.###..###..#....
###.....#.#....###..###..#....#....#....
#.#..#..#.#....#.#..#....#....#....#..#.
#..#..##..####.#..#.#....####.#.....##..


## Alternative using a regex to replace instructions

Replace intructions with 0, converting:

```
addx 2
noop
addx 7
addx -1
```

to:
```
0 2
0
0 7
0 -1
```

In [115]:
import re
from itertools import accumulate

with open(FOLDER/in_file) as f:
    s = f.read()
    s = re.sub(r'(noop|addx)( ([-0-9]+)\n)?', r'0 \2 ', s)
    l = list(accumulate(map(int, s.split()),initial=1))

sum(l[i-1]*i for i in range(20, 221, 40))


14160

In [116]:
for row in range(6):
    for sprite_pos in range(40):
        X = l[40 * row + sprite_pos]
        
        if X + 1 >= sprite_pos >= X - 1:
            print('#', end='')
        else:
            print('.', end='')
    print()



###....##.####.###..###..####.####..##..
#..#....#.#....#..#.#..#.#....#....#..#.
#..#....#.###..#..#.#..#.###..###..#....
###.....#.#....###..###..#....#....#....
#.#..#..#.#....#.#..#....#....#....#..#.
#..#..##..####.#..#.#....####.#.....##..
